In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [35]:
df=pd.read_csv(r"C:\Users\prabh\Downloads\airbnb_project\data\listing_oct_to_sept.csv")

C:\Users\prabh\AppData\Local\Temp\ipykernel_11520\389301483.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"C:\Users\prabh\Downloads\airbnb_project\data\listing_oct_to_sept.csv")


In [36]:

df=df.drop(['has_availability'], axis=1)

In [37]:
df['review_scores_rating'] = df['review_scores_rating'].fillna(0)

In [38]:
# Forward fill missing values in host_since column with values from first review column
df['host_since'] = df['host_since'].fillna(df['first_review'])
# Show null values in host_since column
print(df['host_since'].isnull().sum())

# Drop entire row with null value in host_since column
df = df.dropna(subset=['host_since'])

15


In [39]:
df['property'] = df['property_type'].astype('category').cat.codes
df['neighbourhood'] = df['neighbourhood_cleansed'].astype('category').cat.codes
#df['bathrooms'] = df['bathrooms_text'].astype('category').cat.codes

In [40]:
def to_numerical(df):
    # Remove the dollar signs and commas, then convert to numeric (float64)
    df['price'] = df['price'].str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)
    return df

df = to_numerical(df)

In [41]:
# Calculate average price
average_price_df = df.groupby(['neighbourhood_cleansed', 'property_type', 'accommodates'])['price'].mean().reset_index()
average_price_df = average_price_df.rename(columns={'price': 'average_price'})

# Join with the original DataFrame
df = pd.merge(df, average_price_df, on=['neighbourhood_cleansed', 'property_type', 'accommodates'], how='left')

# Fill missing prices with the average
df['price'] = df['price'].fillna(df['average_price'])

# Drop the average price column
df = df.drop('average_price', axis=1)

In [42]:
def extract_numerical_values(df, columns):
    for column in columns:
        # Use apply with a lambda function to extract the numerical part from the string
        df[column] = df[column].apply(lambda x: float(re.search(r'\d+(?:\.\d+)?', str(x)).group()) if re.search(r'\d+(?:\.\d+)?', str(x)) else None)
    return df

columns = ['bathrooms_text']
df = extract_numerical_values(df, columns)

In [43]:
# Calculate average price
average_bedrooms_df = df.groupby(['neighbourhood_cleansed', 'property_type', 'accommodates'])['bedrooms'].mean().reset_index()
average_bedrooms_df = average_bedrooms_df.rename(columns={'bedrooms': 'average_bedrooms'})

# Join with the original DataFrame
df = pd.merge(df, average_bedrooms_df, on=['neighbourhood_cleansed', 'property_type', 'accommodates'], how='left')

# Fill missing prices with the average
df['bedrooms'] = df['bedrooms'].fillna(df['average_bedrooms'])

# Drop the average price column
df = df.drop('average_bedrooms', axis=1)

In [44]:
# Calculate average beds
average_beds_df = df.groupby(['neighbourhood_cleansed', 'property_type', 'accommodates'])['beds'].mean().reset_index()
average_beds_df = average_beds_df.rename(columns={'beds': 'average_beds'})

# Join with the original DataFrame
df = pd.merge(df, average_beds_df, on=['neighbourhood_cleansed', 'property_type', 'accommodates'], how='left')

# Fill missing beds with the average
df['beds'] = df['beds'].fillna(df['average_beds'])

# Drop the average beds column
df = df.drop('average_beds', axis=1)

In [45]:
# Calculate average bathrooms_text
average_bathrooms_text_df = df.groupby(['neighbourhood_cleansed', 'property_type', 'accommodates'])['bathrooms_text'].mean().reset_index()
average_bathrooms_text_df = average_bathrooms_text_df.rename(columns={'bathrooms_text': 'average_bathrooms_text'})

# Join with the original DataFrame
df = pd.merge(df, average_bathrooms_text_df, on=['neighbourhood_cleansed', 'property_type', 'accommodates'], how='left')

# Fill missing bathrooms_text with the average (mode in this context)
df['bathrooms_text'] = df['bathrooms_text'].fillna(df['average_bathrooms_text'])

# Drop the average bathrooms_text column
df = df.drop('average_bathrooms_text', axis=1)

In [46]:
# Extract the year from the host_since column
df['new_host'] = df['host_since'].str.split('-').str[0].astype(int)

# Replace years less than 2024 with 0 and years greater than or equal to 2024 with 1
df['new_host'] = df['new_host'].apply(lambda x: 0 if x < 2024 else 1)

In [47]:
df=df.drop(['host_since'], axis=1)

In [48]:
# Define the mapping functions
df['host_response_time'] = df['host_response_time'].replace(
  [np.nan, 'within an hour', 'within a few hours', 'within a day', 'a few days or more'],
  [0, 1, 2, 3, 4])

def map_response_rate(rate):
    if pd.isna(rate):
        return 0
    rate = int(str(rate).strip('%'))
    if 90 <= rate <= 100:
        return 4
    elif 70 <= rate < 90:
        return 3
    elif 50 <= rate < 70:
        return 2
    elif 0 <= rate < 50:
        return 1
    else:
        return 0

def map_acceptance_rate(rate):
    if pd.isna(rate):
        return 0
    rate = int(str(rate).strip('%'))
    if 90 <= rate <= 100:
        return 4
    elif 70 <= rate < 90:
        return 3
    elif 50 <= rate < 70:
        return 2
    elif 0 <= rate < 50:
        return 1
    else:
        return 0

# Apply the mapping functions to the DataFrame

df['host_response_rate'] = df['host_response_rate'].apply(map_response_rate)
df['host_acceptance_rate'] = df['host_acceptance_rate'].apply(map_acceptance_rate)




C:\Users\prabh\AppData\Local\Temp\ipykernel_11520\2672766150.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['host_response_time'] = df['host_response_time'].replace(


In [49]:
# Adding 3 columns to create the new column
df['host_response_score'] = df['host_response_time'] + df['host_response_rate'] + df['host_acceptance_rate']

# Dropping the 3 columns used for the addition
df = df.drop(columns=['host_response_time', 'host_response_rate','host_acceptance_rate' ])

In [50]:
# For host location, changing to binary column, with 0 as outside canada, and 1 as inside canada
def within_canada(location):
    if 'Canada' in str(location):
        return 1
    else:
        return 0

# Applying the within canada function on host location
df['host_location'] = df['host_location'].apply(within_canada)

In [51]:
#Changes t to 1 and f to 0 in host_is_superhost column
df['host_is_superhost'] = df['host_is_superhost'].str.contains('t', na=False).astype(int)
df['instant_bookable'] = df['instant_bookable'].str.contains('t', na=False).astype(int)

In [52]:
#Dropping host_neighborhood and neighborhood columns, and keeping only neighborhood_cleansed column.
df= df.drop(columns=['neighbourhood', 'host_neighbourhood'], axis=1)

In [53]:
#Dropping host_verifications column because it has not much importance in our predictions
df= df.drop(columns=['host_verifications'], axis=1)

In [54]:
# Define the list of columns to update with the 'contains text' logic
columns_to_update = [
    'host_has_profile_pic', 
    'host_identity_verified', 
    'host_about',
    'neighborhood_overview',
    'description'
]

# Iterate through each column in the list
for column in columns_to_update:
    # Apply a lambda function to each value in the column:
    #   - If the value is truthy (i.e., not null, not empty string, not False) -> assign 1
    #   - Otherwise (null, empty string, False) -> assign 0
    df[column] = df[column].apply(lambda x: 
        # Check if the value is truthy (and not an empty string after stripping whitespace)
        1 if x and str(x).strip() else 0
    )

In [55]:
# Adding 5 host related columns to create the new column host_score
df['host_score'] = df['host_location'] + df['host_is_superhost'] + df['host_has_profile_pic'] + df['host_identity_verified'] + df['host_about']

# Adding 2 property description related columns to create the new column property_description_score
df['property_description_score'] = df['neighborhood_overview'] + df['description']

# Dropping the 6 columns used for the addition
df = df.drop(columns=['host_location','host_is_superhost', 'host_has_profile_pic','host_identity_verified', 'host_about',
                      'neighborhood_overview' ,'description'])

In [56]:
#corr_matrix = df[['bathrooms', 'beds', 'bedrooms', 'accommodates']].corr()

# Print the correlation matrix
#print(corr_matrix)

df = df.drop(['room_type', 'source','bathrooms'], axis=1)
#  'bedrooms', 'beds',

property_type, room_type, accommodates, bathrooms, bathrooms_text, bedrooms, beds     
If the model's accuracy score is low and any of these columns has high feature importance, we will come back and revisit the choice of columns, feature engg feature selection and encoding to further improve the model metrices.

We believe that the columns number_of_reviews_l30d, number_of_reviews_ltm, reviews_per_month information overlaps with the number_of_reviews column so we will be dropping but if needed we will revisit to check if they can be used to improve the metrices.

The columns first_reviews and last_review aren't that important to predict price but revisit if needed.

In [57]:
#df = df.drop(['property_type', 'neighbourhood_cleansed'], axis=1)

df = df.drop(['first_review', 'last_review', 'calendar_last_scraped', 'number_of_reviews_ltm',
               'number_of_reviews_l30d', 'license','reviews_per_month'], axis=1)

In [58]:
df['count_amenities'] = df['amenities'].apply(len)
df['count_amenities'] = df['count_amenities'].astype(int)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248508 entries, 0 to 248507
Data columns (total 29 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              248508 non-null  int64  
 1   host_id                         248508 non-null  int64  
 2   neighbourhood_cleansed          248508 non-null  object 
 3   property_type                   248508 non-null  object 
 4   accommodates                    248508 non-null  int64  
 5   bathrooms_text                  248376 non-null  float64
 6   bedrooms                        246462 non-null  float64
 7   beds                            248308 non-null  float64
 8   amenities                       248508 non-null  object 
 9   price                           248470 non-null  float64
 10  minimum_nights                  248508 non-null  int64  
 11  maximum_nights                  248508 non-null  int64  
 12  minimum_nights_a

In [60]:
# Convert month column to the desired format (YYYY-MM-DD)
df['month'] = pd.to_datetime(df['month'], format='%Y-%m-%d')

In [61]:
df = df.drop(['availability_365', 'availability_60', 'availability_90', 'host_id'], axis=1)
#amenities

In [62]:
#df = df.drop(['id'], axis=1)

In [63]:
df['data_year'] = df['month'].dt.year
df['data_month'] = df['month'].dt.month

# Drop the original datetime column
df = df.drop(columns=['month'])

In [64]:
df = df.dropna(subset=['bedrooms','bathrooms_text', 'beds'])

In [65]:
import pandas as pd
import ast
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_selection import SelectKBest, chi2


In [67]:
# Ensure 'amenities' column has no NaN values
df['amenities'] = df['amenities'].fillna('[]')

# Parse the amenities column
def parse_amenities(entry):
    try:
        # Use ast.literal_eval to safely parse the string to a list
        amenities_list = ast.literal_eval(entry)
        return amenities_list
    except (ValueError, SyntaxError):
        # Handle parsing errors
        return []
#df.dropna(subset=['amenities_list'], inplace=True)

In [68]:
def normalize_amenities(amenities_list):
    normalized = []
    for amenity in amenities_list:
        amenity = amenity.lower().strip()
        amenity = amenity.replace(' ', '_').replace('"', '').replace("'", '')
        if 'translation_missing' not in amenity and amenity != '':
            normalized.append(amenity)
    return normalized


In [69]:
# Apply parsing and normalization functions
df['amenities_list'] = df['amenities'].apply(lambda x: normalize_amenities(parse_amenities(x)))


In [70]:
df# Flatten the list of all amenities
all_amenities = [amenity for sublist in df['amenities_list'] for amenity in sublist]

# Count the occurrences of each amenity
amenity_counts = Counter(all_amenities)

# Calculate total number of listings
total_listings = len(df)

# Define frequency thresholds (e.g., between 5% and 95%)
min_freq = 0.02
max_freq = 0.98

# Identify frequent amenities
frequent_amenities = [
    amenity for amenity, count in amenity_counts.items()
    if min_freq <= (count / total_listings) <= max_freq
]

# Filter the amenities_list to keep only frequent amenities
df['amenities_list'] = df['amenities_list'].apply(
    lambda x: [amenity for amenity in x if amenity in frequent_amenities]
)


In [71]:
# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Fit and transform the amenities_list
amenities_encoded = mlb.fit_transform(df['amenities_list'])

# Create a DataFrame with the encoded amenities
amenities_df = pd.DataFrame(amenities_encoded, columns=mlb.classes_)


In [72]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression

# Define the number of top amenities to select
top_n = 100  # Adjust as needed

# Initialize the selector with mutual_info_regression
selector = SelectKBest(score_func=mutual_info_regression, k=top_n)

# Fit the selector to the data
amenities_selected = selector.fit_transform(amenities_df, df['price'])

# Get the indices of selected features
selected_indices = selector.get_support(indices=True)

# Get the names of the selected amenities
selected_amenities = [mlb.classes_[i] for i in selected_indices]

# Create a DataFrame with the selected amenities
amenities_df_selected = amenities_df[selected_amenities]


In [73]:
# Combine with original data
df = df.reset_index(drop=True)
amenities_df_selected = amenities_df_selected.reset_index(drop=True)
df = pd.concat([df, amenities_df_selected], axis=1)

In [74]:
# Drop unnecessary columns
df = df.drop(columns=['amenities', 'amenities_list'])

In [80]:
df.to_csv(r"C:\Users\prabh\Downloads\airbnb_project\data\listing_1_With_Amenities.csv", index=False)